In [14]:
import numpy as np
import pandas as pd
import csv
import pandasql as ps
import sqlite3
import os

import datetime
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [15]:
report = pd.read_csv('/home/xuranzeng/project/data/aligned_sample.csv')
report.drop(report.iloc[:, 0:1], inplace = True, axis = 1)

In [16]:
report = report[['date','Zone','file','num_bpc','Date2','Arrest','Domestic','Primary Type']]

In [17]:
report.head()

,date,Zone,file,num_bpc,Date2,Arrest,Domestic,Primary Type
0,2018_08_18,1,201808172356,97,2018-08-18 00:05:00,0,1,BATTERY
1,2018_08_18,1,201808180126,110,2018-08-18 01:30:00,1,0,BATTERY
2,2018_08_18,1,201808180126,110,2018-08-18 01:30:00,0,1,OFFENSE INVOLVING CHILDREN
3,2018_08_18,1,201808180156,171,2018-08-18 02:00:00,1,1,BATTERY
4,2018_08_18,1,201808180156,171,2018-08-18 02:00:00,0,0,CRIMINAL DAMAGE


In [18]:
# label violent and property crime
violent_crime = ["HOMICIDE","ASSAULT","BATTERY"]
property_crime = ["BURGLARY","THEFT","MOTOR VEHICLE THEFT"]

report['Violent'] = np.where(report['Primary Type'].isin(violent_crime),True,False)
report['Property'] = np.where(report['Primary Type'].isin(property_crime),True,False)

report['Total'] = True

In [20]:
# groupby summary (day)
report['file'] = pd.to_numeric(report['file'])
report = report.sort_values(by=['date','file','Zone'])
report_sum = report.groupby(['date',"file","Zone"]).sum()
#report_sum.to_csv('/home/xuranzeng/project/data/crimes_data_zone_simplified_sample_summary_day.csv')


In [21]:
# get audio file name
file = report_sum.reset_index()
file['zonename'] = 'Zone'+file['Zone'].astype(str)
base_path='/project/graziul/data/'
for i in range(0,len(file)):
    
    
    feed=file.loc[i,'zonename']
    date=file.loc[i,'date']
    filepath=base_path+feed+'/'+date
    infiles = os.listdir(filepath)

    l = (item for item in infiles if item.endswith(".mp3") and item.startswith(file.loc[i,'file'].astype(str)))


    #file['mp3path'] = base_path+feed+'/'+date+'/'+list(l)[0]
    file.loc[i,'mp3path'] = list(l)[0]
    
file['path'] = base_path+file['zonename'] +'/'+file['date']+'/'+file['mp3path']

In [22]:
file.shape

(9168, 12)

In [23]:
file.head(2)

,date,file,Zone,num_bpc,Arrest,Domestic,Violent,Property,Total,zonename,mp3path,path
0,2018_08_18,201808172356,1,97,0,1,1,0,1,Zone1,201808172356-479346-27730.mp3,/project/graziul/data/Zone1/2018_08_18/2018081...
1,2018_08_18,201808180126,1,220,1,1,1,0,2,Zone1,201808180126-214247-27730.mp3,/project/graziul/data/Zone1/2018_08_18/2018081...


In [24]:
file[file['path'].isnull()]

,date,file,Zone,num_bpc,Arrest,Domestic,Violent,Property,Total,zonename,mp3path,path


In [25]:
# TEO
from teo_cb_auto_env import TeagorExtractor

import pydub
import librosa
from scipy import signal
from mutagen.mp3 import MP3
import numpy as np
from numpy import sign, trapz
from teager_py import Teager
import statsmodels.api as sm
import wave
import statistics
from pydub import AudioSegment

t = TeagorExtractor()

In [ ]:
for i in range(0,len(file)):
    filepath = file.loc[i,'path']
    teo_cb_auto_env = t.process_mp3(filepath,3)
    file.loc[i,'band1'] = statistics.mean(teo_cb_auto_env[:,0])
    file.loc[i,'band16'] = statistics.mean(teo_cb_auto_env[:,-1])
    print('processing:',i)

Computing TEO Features ...
0 610
processing: 0
Computing TEO Features ...
0 610
processing: 1
Computing TEO Features ...
0 610
processing: 2
Computing TEO Features ...
0 610
processing: 3
Computing TEO Features ...
0 610
processing: 4
Computing TEO Features ...
0 610
processing: 5
Computing TEO Features ...
0 610
processing: 6
Computing TEO Features ...
0 610
processing: 7
Computing TEO Features ...
0 610
processing: 8
Computing TEO Features ...
0 610
processing: 9
Computing TEO Features ...
0 610
processing: 10
Computing TEO Features ...
0 610
processing: 11
Computing TEO Features ...
0 610
processing: 12
Computing TEO Features ...
0 610
processing: 13
Computing TEO Features ...
0 610
processing: 14
Computing TEO Features ...
0 610
processing: 15
Computing TEO Features ...
0 610
processing: 16
Computing TEO Features ...
0 610
processing: 17
Computing TEO Features ...
0 610
processing: 18
Computing TEO Features ...
0 610
processing: 19
Computing TEO Features ...
0 610
processing: 20
Co

In [ ]:
file.to_csv('/home/xuranzeng/project/data/0105_TEO.csv')

In [ ]:
file.head(5)

In [11]:
filepath = file.loc[0,'path']
filepath

'/project/graziul/data/Zone1/2018_08_18/201808172356-479346-27730.mp3'

In [13]:
teo_cb_auto_env = t.process_mp3(filepath,1)

/home/xuranzeng/.local/lib/python3.8/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Computing TEO Features ...
0 1830


/software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:655: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  warnings.warn(
/software/python-anaconda-2020.11-el8-x86_64/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:665: FutureWarning: fft=True will become the default after the release of the 0.12 release of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(


In [ ]:
band1 = statistics.mean(teo_cb_auto_env[:,0])

In [ ]:
band16 = statistics.mean(teo_cb_auto_env[:,-1])